In [ ]:
import statsmodels.api as sm 
import pandas as pd  
import numpy as np
import os

In [ ]:
dem_data = pd.read_csv('dem_data.csv') #file with symptom scores, gender, medication information etc

# group information containing subject ids
controls = np.loadtxt('control_subs.txt', dtype= str)
patients = np.loadtxt('patient_subs.txt', dtype= str)
all_subs = np.concatenate([controls, patients])

In [ ]:
os_sep = os.path.abspath(os.sep)
wd = os.getcwd()
dfs = os.path.join(os_sep, wd, 'opensmile', 'egemaps_summary_turns_zero_filtered') #feature dataframes
summary_dir = os.path.join(os_sep, wd, 'group_level') #synchrony dataframes for all participants

In [ ]:
# load synchrony dataframes
pitch = pd.read_csv(os.path.join(summary_dir, 'F0semitoneFrom27.5Hz_sma3nz_amean_summary.csv'), sep = ';', index_col = [0])
loudness = pd.read_csv(os.path.join(summary_dir, 'loudness_sma3_amean_summary.csv'), sep = ';', index_col = [0])
syll = pd.read_csv(os.path.join(summary_dir, 'VoicedSegmentsPerSec_summary.csv'), sep = ';', index_col = [0])
pause = pd.read_csv(os.path.join(summary_dir, 'MeanUnvoicedSegmentLength_summary.csv'), sep = ';', index_col = [0])
pitch_var = pd.read_csv(os.path.join(summary_dir, 'F0semitoneFrom27.5Hz_sma3nz_stddevNorm_summary.csv'), sep = ';', index_col = [0])

#### Add accommodation coefficients as predictors for the model

In [ ]:
#make dataframe containing the predictors for all acoustic features

feature_df = pd.DataFrame([pitch['r'], loudness['r'], syll['r'], pause['r'], pitch_var['r']]).T
feature_df.columns = ['pitch', 'loudness', 'syll_rate', 'pause_dur', 'pitch_var']
feature_df.index = pitch['soundname']

In [ ]:
interviewers = dem_data.loc[all_subs]['Interviewer'] #who conducted the interview
yoe = dem_data.loc[all_subs]['YOE_handmatig'].sort_index() #years of education

In [ ]:
from sklearn.preprocessing import LabelEncoder

#encode interviewers numerically
le = LabelEncoder()
interviewers_encod = le.fit_transform(interviewers)
interviewers_encod[interviewers.isna()] =  -99999 #preserve invalid entries

#### Add YOE as predictor

The interviewer can be added as a predictor as well, but this information is not available for all subjects so it comes with a great loss of data.

In [ ]:
feature_df['yoe'] = yoe

## optionally add interviewer as predictor
#feature_df['interviewer'] = interviewers_encod
#feature_df.replace(-99999, np.nan, inplace= True)
#feature_df = feature_df.dropna()

In [ ]:
#print predictors
feature_df

In [ ]:
# encode target variable "group" numerically
group_encoding = []

for sub in feature_df.index:
    if sub in controls:
        group_encoding.append(0)
    else:
        group_encoding.append(1)

#### Add target variable

In [ ]:
group_df = pd.DataFrame(group_encoding, columns = ['group'])
group_df.index = feature_df.index

#### Assert indipendence of predictors

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

X = feature_df  # independent variables
y = group_df   # dependent variables
X_intcpt = sm.add_constant(X)

# Variance Inflation Factor (VIF) to analyze multicolinearity
# Between 1 and 5 = moderately correlated.
pd.DataFrame({'variables':X_intcpt.columns[1:], 'VIF':[variance_inflation_factor(X_intcpt.values, i+1) for i in range(len(X_intcpt.columns[1:]))]})

#### Split the data in train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (confusion_matrix, roc_auc_score) 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

#### Using Recursive feature elimination to determine the optimal amount of features for training

In [ ]:
#This is to select 1 variable: can be changed and checked in model for accuracy
min_features_to_select = 1

rfe_mod =  RFECV(estimator = LogisticRegression(), step=1, scoring = 'roc_auc')

rfe_fit = rfe_mod.fit(X_train, np.array(y_train).flatten())

# (i.e., estimated best) features are assigned rank 1.
print(rfe_fit.ranking_)

rfe_features = X.columns[rfe_fit.support_]
print(rfe_features)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("AUC score")
plt.plot(range(min_features_to_select,
               len(rfe_fit.grid_scores_) + min_features_to_select),
         rfe_fit.grid_scores_)
plt.show()

#### Train the model using stratified 5-fold cross-validation

In [ ]:
cv = StratifiedKFold(n_splits = 5, shuffle = True)

X_arr = X.to_numpy()
y_arr = y.to_numpy().flatten()

In [ ]:
log_sklearn = LogisticRegression()

tprs = [] #true positive rate
aucs = [] #auc scores for each fold

mean_fpr = np.linspace(0, 1, 100)
plt.figure(figsize=(10,10))
i = 1

for train, test in cv.split(X_arr, y_arr):
    
    log_fit = log_sklearn.fit(X_arr[train], y_arr[train])
    
    yhat_proba = log_fit.predict_proba(X_arr[test])
    
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y_arr[test], yhat_proba[:, 1])
    
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1

#### Plot the performance

In [ ]:
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate',fontsize=18)
plt.ylabel('True Positive Rate',fontsize=18)
plt.title('Cross-Validation ROC',fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})

#### Train and evaluate full model

In [ ]:
log_fit = log_sklearn.fit(X_train, np.array(y_train).flatten())

yhat = log_sklearn.predict(X_test)

#### Plot performance

In [ ]:
import seaborn as sns

cm = confusion_matrix(y_test['group'], yhat)

fig, ax = plt.subplots(figsize= (7, 7))

sns.heatmap(cm, annot=True, cmap = 'Blues', ax = ax)

ax.set_yticklabels(['Control', 'Patient'], fontsize = 13)
ax.set_xticklabels(['Control', 'Patient'], fontsize = 13)
ax.set_title('Confusion Matrix', fontsize = 15)

#### Analyze the false positives

In [ ]:
real_vs_prediction = y_test
real_vs_prediction['yhat'] = yhat

In [ ]:
#identify which patients were correctly or incorrectly classified
incorrect_ident = real_vs_prediction.loc[(real_vs_prediction['group'] == 1) & (real_vs_prediction['yhat'] == 0)]
correct_ident = real_vs_prediction.loc[(real_vs_prediction['group'] == 1) & (real_vs_prediction['yhat'] == 1)]

#### compare the panss scores between the two groups

In [ ]:
panss_cols = ['PANSS_datum','PANSS_P1','PANSS_P2','PANSS_P3','PANSS_P4',
              'PANSS_P5','PANSS_P6','PANSS_P7','PANSS_N1','PANSS_N2','PANSS_N3',
              'PANSS_N4','PANSS_N5','PANSS_N6','PANSS_N7','PANSS_G1','PANSS_G2',
              'PANSS_G3','PANSS_G4','PANSS_G5','PANSS_G6','PANSS_G7','PANSS_G8',
              'PANSS_G9','PANSS_G10','PANSS_G11','PANSS_G12','PANSS_G13','PANSS_G14',
              'PANSS_G15','PANSS_G16','PANSS_remission','PANSS_totaal','PANSS_positive',
              'PANSS_negative','PANSS_general','PANSS_Positive_factor','PANSS_negative_factor',
              'PANSS_disorganized_factor','PANSS_excited_factor','PANSS_depressed_factor']

In [ ]:
panss_incorrect = dem_data.loc[incorrect_ident.index][panss_cols]
panss_correct = dem_data.loc[correct_ident.index][panss_cols]

In [ ]:
#filter out the participants with missing scores
panss_correct_filt = panss_correct.loc[(panss_correct['PANSS_remission'] != 'geen PANSS') & (panss_correct['PANSS_totaal'] < 1000)]

In [ ]:
panss_incorrect_filt = panss_incorrect.loc[(panss_incorrect['PANSS_remission'] != 'geen PANSS') & (panss_incorrect['PANSS_totaal'] < 1000)]

#### Print scores of each group

In [ ]:
incorr_neg = panss_incorrect_filt[['PANSS_N1','PANSS_N2','PANSS_N3','PANSS_N4','PANSS_N5','PANSS_N6','PANSS_N7']]
incorr_neg

In [ ]:
corr_neg = panss_correct_filt[['PANSS_N1','PANSS_N2','PANSS_N3','PANSS_N4','PANSS_N5','PANSS_N6','PANSS_N7']]
corr_neg

#### Transform the data for easier plotting

In [ ]:
corr_neg_melt = corr_neg.melt(var_name='symptom', value_name='score')
corr_neg_melt['soundname'] = np.concatenate([[corr_neg.index]] * 7).flatten()
corr_neg_melt['kind'] = 'correctly_identified'

In [ ]:
incorr_neg_melt = incorr_neg.melt(var_name='symptom', value_name='score')
incorr_neg_melt['soundname'] = np.concatenate([[incorr_neg.index]] * 7).flatten()
incorr_neg_melt['kind'] = 'incorrectly_identified'

In [ ]:
both = pd.concat([corr_neg_melt, incorr_neg_melt])

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize= (13, 7))

#plot scores of correctly identified patients
sns.stripplot(x="symptom", y="score", data = corr_neg_melt, 
              marker = '^', size = 7, linewidth=0.5, color = '#91bfdb',  ax = ax)

#plot scores of incorrectly identified patients
sns.stripplot(x="symptom", y="score", data = incorr_neg_melt, 
              marker = 'D', size = 7, linewidth=0.5, color = '#fc8d59',  ax = ax)

sns.pointplot(x="symptom", y="score", ci= None, capsize=.2, hue = 'kind', data = both)

ax.set_ylabel('PANSS Score', size = 13)
ax.set_xlabel('')

ax.set_xticklabels(['Blunted Affect', 'Emotional Withdrawal', 
                    'Poor Rapport', 'Social Withdrawal', 'Difficulty Abstract Thinking',
                    'Flow of conversation', 'Stereotyped Thinking'], rotation = 45, ha='right', fontsize = 13)


corr_patch = mpatches.Patch(color = '#91bfdb', label='Correctly Identified Patients')
incorr_patch = mpatches.Patch(color = '#fc8d59', label='Incorrectly Identified Patients')

plt.legend(handles=[corr_patch, incorr_patch], fontsize = 13)
plt.title('PANSS Scores of Negative Symptoms', fontsize = 15)

plt.tight_layout()